In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datacon-22/Processed_data/bookings.csv
/kaggle/input/datacon-22/Processed_data/hotels_data.csv
/kaggle/input/datacon-22/Processed_data/bookings_data.csv
/kaggle/input/datacon-22/Processed_data/train_data.csv
/kaggle/input/datacon-22/Processed_data/customer_data.csv
/kaggle/input/datacon-22/Processed_data/sample_submission_5.csv
/kaggle/input/datacon-22/Processed_data/payments_data.csv
/kaggle/input/27-11-filtering-columns-date-new/__results__.html
/kaggle/input/27-11-filtering-columns-date-new/__notebook__.ipynb
/kaggle/input/27-11-filtering-columns-date-new/__output__.json
/kaggle/input/27-11-filtering-columns-date-new/custom.css
/kaggle/input/27-11-filtering-columns-date-new/full_data_to_use_27-11.csv
/kaggle/input/27-11-filtering-columns-date-new/__results___files/__results___80_0.png
/kaggle/input/27-11-filtering-columns-date-new/__results___files/__results___50_1.png
/kaggle/input/27-11-filtering-columns-date-new/__results___files/__results___9_0.png
/kaggle/input/27

In [2]:
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/kaggle/input/27-11-filtering-columns-date-new/full_data_to_use_27-11.csv')
df.head(2)

,booking_id,rating_score,set,booking_status,country,payment_type_mode,payment_type_count,payment_sequential_sum,payment_installments_count,payment_installments_sum,...,booking_checkin_customer_date_weekday,booking_checkin_customer_date_hour,payment_value_sum_log2,price_sum_log2,agent_fees_sum_log2,hotel_description_length_mean_log2,booking_approved_minus_created_days,booking_approved_minus_created_hours_log2,booking_checkin_minus_created_days_log2,booking_checkin_minus_created_hours_log2
0,87114b39712e6da79b0a377eb73fc7af,4,training,5,5,6.0,1.0,1.0,1.0,8.0,...,2.0,18.0,8.634303,8.631381,4.773996,9.844834,0.0,-1.372912,2.608809,3.979684
1,81e283fa7e4f11123a3fb894f18e6bfb,5,training,5,0,6.0,1.0,1.0,1.0,10.0,...,2.0,16.0,10.380840,10.327616,7.069853,9.234426,0.0,2.299147,4.177918,5.551460


In [4]:
from sklearn import model_selection

In [5]:
df_train = df.loc[df['set'] == 'training']
df_testing = df.loc[df['set'] == 'testing']

In [6]:
df = df_train.copy() # likhne ko easy jayega aage

In [7]:
# we create a new column called kfold and fill it with -1
df["kfold"] = -1

# the next step is to randomize the rows of the data
df = df.sample(frac=1).reset_index(drop=True)

# fetch targets
y = df.rating_score.values

# initiate the kfold class from model_selection module
kf = model_selection.StratifiedKFold(n_splits=5)

# fill the new kfold column
for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f

# save the new csv with kfold column
df.to_csv("data_train_folds.csv", index=False)

In [8]:
df = pd.read_csv('/kaggle/working/data_train_folds.csv') # train
df_testing = pd.read_csv('/kaggle/input/27-11-filtering-columns-date-new/full_data_to_use_27-11.csv')
df_testing = df_testing.loc[df_testing['set'] == 'testing']

In [9]:
df.head(2)

,booking_id,rating_score,set,booking_status,country,payment_type_mode,payment_type_count,payment_sequential_sum,payment_installments_count,payment_installments_sum,...,booking_checkin_customer_date_hour,payment_value_sum_log2,price_sum_log2,agent_fees_sum_log2,hotel_description_length_mean_log2,booking_approved_minus_created_days,booking_approved_minus_created_hours_log2,booking_checkin_minus_created_days_log2,booking_checkin_minus_created_hours_log2,kfold
0,d056e978c305b3134cf612e5286ca4b6,5,training,5,1,6.0,1.0,1.0,1.0,1.0,...,16.0,5.242984,5.006410,2.978196,9.701771,0.0,-1.640104,2.035624,3.373893,0
1,1623c2fc6be250ca6f1c55f7ccb17dea,5,training,5,2,6.0,1.0,1.0,1.0,4.0,...,19.0,6.369990,6.242005,3.595742,9.213742,0.0,-1.555215,2.608809,4.140427,0


In [10]:
df['kfold'].value_counts()

0    10000
1    10000
2    10000
3    10000
4    10000
Name: kfold, dtype: int64

In [11]:
df.drop(["booking_id", 'set'], axis=1, inplace = True)

In [12]:
df['rating_score'].value_counts()

5    28961
4     9576
1     5755
3     4167
2     1541
Name: rating_score, dtype: int64

In [13]:
# when predicting do
df_testing.drop(["booking_id", 'set'], axis=1, inplace = True)

## Simple Decision Tree Classifier

In [14]:
import joblib
from sklearn import metrics
from sklearn import tree

In [15]:
def run_DT(fold):
        
    # training data is where kfold is not equal to provided fold
    # also, note that we reset the index
    df_train = df[df.kfold != fold].reset_index(drop=True)
    
    # validation data is where kfold is equal to provided fold
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    x_train = df_train.drop(["rating_score", "kfold"], axis=1).values
    y_train = df_train.rating_score.values
    
    x_valid = df_valid.drop(["rating_score", "kfold"], axis=1).values
    y_valid = df_valid.rating_score.values
    
    clf = tree.DecisionTreeClassifier(max_depth = 8, min_samples_split = 15, 
                                      min_samples_leaf = 5, class_weight = None,
                                     criterion = 'entropy')
    
    clf.fit(x_train, y_train)
    print(clf.get_depth(), clf.get_n_leaves())
    
    # create predictions for validation samples
    preds = clf.predict(x_valid)

    accuracy = metrics.accuracy_score(y_valid, preds)
    mse = metrics.mean_squared_error(y_valid, preds)
    
    print(f"Fold={fold}, Accuracy={accuracy}, MSE = {mse}") #", roc_auc = {roc_auc}")
    
    # save the model
    joblib.dump(clf, f"/kaggle/working/dt_{fold}.bin")

In [16]:
 run_DT(fold=0)
 run_DT(fold=1)
 run_DT(fold=2)
 run_DT(fold=3)
 run_DT(fold=4)

8 158
Fold=0, Accuracy=0.612, MSE = 2.0728
8 179
Fold=1, Accuracy=0.6112, MSE = 2.0666
8 179
Fold=2, Accuracy=0.6117, MSE = 2.0669
8 186
Fold=3, Accuracy=0.6131, MSE = 2.0022
8 173
Fold=4, Accuracy=0.6095, MSE = 2.0798


## RF

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
def run_RF(fold):
        
    # training data is where kfold is not equal to provided fold
    # also, note that we reset the index
    df_train = df[df.kfold != fold].reset_index(drop=True)
    
    # validation data is where kfold is equal to provided fold
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    x_train = df_train.drop(["rating_score", "kfold"], axis=1).values
    y_train = df_train.rating_score.values
    
    x_valid = df_valid.drop(["rating_score", "kfold"], axis=1).values
    y_valid = df_valid.rating_score.values
    
    clf = RandomForestClassifier()
    clf.fit(x_train, y_train)
        
    preds = clf.predict(x_valid)

    # calculate & print accuracy
    accuracy = metrics.accuracy_score(y_valid, preds)
    mse = metrics.mean_squared_error(y_valid, preds)
    
    print(f"Fold={fold}, Accuracy={accuracy}, MSE = {mse}")
    
    joblib.dump(clf, f"/kaggle/working/rf_{fold}.bin")

In [19]:
 run_RF(fold=0)
 run_RF(fold=1)
 run_RF(fold=2)
 run_RF(fold=3)
 run_RF(fold=4)

Fold=0, Accuracy=0.6134, MSE = 2.0372
Fold=1, Accuracy=0.6134, MSE = 2.0491
Fold=2, Accuracy=0.6133, MSE = 2.0397
Fold=3, Accuracy=0.6168, MSE = 1.9545
Fold=4, Accuracy=0.6127, MSE = 2.0427


## Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression

In [21]:
def run_LR(fold):
        
    # training data is where kfold is not equal to provided fold
    # also, note that we reset the index
    df_train = df[df.kfold != fold].reset_index(drop=True)
    
    # validation data is where kfold is equal to provided fold
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    x_train = df_train.drop(["rating_score", "kfold"], axis=1).values
    y_train = df_train.rating_score.values
    
    x_valid = df_valid.drop(["rating_score", "kfold"], axis=1).values
    y_valid = df_valid.rating_score.values
    
    clf = LogisticRegression(solver = 'liblinear') #'newton-cg’, ‘lbfgs’ -- fails to converge, , ‘sag’, ‘saga’
    clf.fit(x_train, y_train)
        
    preds = clf.predict(x_valid)

    # calculate & print accuracy
    accuracy = metrics.accuracy_score(y_valid, preds)
    mse = metrics.mean_squared_error(y_valid, preds)
    
    print(f"Fold={fold}, Accuracy={accuracy}, MSE = {mse}")
    
    joblib.dump(clf, f"/kaggle/working/lr_{fold}.bin")

In [22]:
 run_LR(fold=0)
 run_LR(fold=1)
 run_LR(fold=2)
 run_LR(fold=3)
 run_LR(fold=4)

Fold=0, Accuracy=0.5989, MSE = 2.3235
Fold=1, Accuracy=0.6004, MSE = 2.296
Fold=2, Accuracy=0.6041, MSE = 2.2415
Fold=3, Accuracy=0.6027, MSE = 2.2748
Fold=4, Accuracy=0.6017, MSE = 2.2884


## Try to make preds (from each model trained on little folds) -- later will have to retrain on all data?

In [23]:
trained_models_list = []
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        if 'bin' in filename:
            model = os.path.join(dirname, filename)
            print(model)
            trained_models_list.append(model)

/kaggle/working/dt_0.bin
/kaggle/working/dt_2.bin
/kaggle/working/lr_2.bin
/kaggle/working/lr_4.bin
/kaggle/working/dt_4.bin
/kaggle/working/rf_0.bin
/kaggle/working/rf_3.bin
/kaggle/working/lr_0.bin
/kaggle/working/rf_2.bin
/kaggle/working/lr_3.bin
/kaggle/working/dt_3.bin
/kaggle/working/rf_4.bin
/kaggle/working/lr_1.bin
/kaggle/working/rf_1.bin
/kaggle/working/dt_1.bin


In [24]:
# clf = joblib.load('/kaggle/working/rf_1.bin')
# clf
models = [joblib.load(model_path) for model_path in trained_models_list]

In [25]:
all_preds = np.array([clf.predict(df_testing.drop("rating_score", axis=1).values) for clf in models])
all_preds

array([[5, 5, 5, ..., 5, 5, 5],
       [5, 5, 5, ..., 5, 5, 5],
       [5, 5, 5, ..., 5, 5, 5],
       ...,
       [5, 5, 5, ..., 5, 5, 5],
       [5, 5, 5, ..., 5, 5, 5],
       [5, 5, 5, ..., 5, 5, 5]])

In [26]:
all_preds.shape

(15, 49079)

In [27]:
mean_preds = np.mean(all_preds, axis = 0)
np.unique(mean_preds)

array([1.        , 1.06666667, 1.13333333, 1.2       , 1.26666667,
       1.33333333, 1.4       , 1.46666667, 1.53333333, 1.6       ,
       1.66666667, 1.73333333, 1.8       , 1.86666667, 1.93333333,
       2.        , 2.06666667, 2.13333333, 2.2       , 2.26666667,
       2.33333333, 2.4       , 2.46666667, 2.53333333, 2.6       ,
       2.66666667, 2.73333333, 2.8       , 2.86666667, 2.93333333,
       3.        , 3.06666667, 3.13333333, 3.2       , 3.26666667,
       3.33333333, 3.4       , 3.46666667, 3.53333333, 3.6       ,
       3.66666667, 3.73333333, 3.8       , 3.86666667, 3.93333333,
       4.        , 4.06666667, 4.13333333, 4.2       , 4.26666667,
       4.33333333, 4.4       , 4.46666667, 4.53333333, 4.6       ,
       4.66666667, 4.73333333, 4.8       , 4.86666667, 4.93333333,
       5.        ])

In [28]:
from scipy import stats

In [29]:
mode_preds = stats.mode(all_preds)
mode_preds

ModeResult(mode=array([[5, 5, 5, ..., 5, 5, 5]]), count=array([[15, 15, 15, ..., 15, 15, 15]]))

In [30]:
mode_preds.mode

array([[5, 5, 5, ..., 5, 5, 5]])

In [31]:
sub = pd.read_csv('/kaggle/input/datacon-22/Processed_data/sample_submission_5.csv')
sub.head()

,booking_id,rating_score
0,796b98fdf73dbeba33a548910a1c6147,5
1,b201a9f2ecdecbb34bed034bf9e4b658,5
2,97b385a9be170737859d3511b658677c,5
3,366873cd26c5724d1dcb18dcdf73be66,5
4,c3a2e87348811bcfdf22be48aa0d2dce,5


In [32]:
sub.rating_score = mode_preds.mode.T

In [33]:
sub

,booking_id,rating_score
0,796b98fdf73dbeba33a548910a1c6147,5
1,b201a9f2ecdecbb34bed034bf9e4b658,5
2,97b385a9be170737859d3511b658677c,5
3,366873cd26c5724d1dcb18dcdf73be66,5
4,c3a2e87348811bcfdf22be48aa0d2dce,5
...,...,...
49074,b221f926ba560907d8f7f62f01ec64c6,1
49075,2977fc6ce3d412c60000bdcdda88e6b0,5
49076,2269711bd1eaf7eed8fd32a7c95d5cb1,5
49077,245723df7cb52772a34416b41b1acd45,5


In [34]:
sub.to_csv('submission_1_train_on_folds_no_tuning.csv',index=False)